In [26]:
import numpy as np
import pandas as pd

import redis

In [27]:
# Connect to Redis Client
#redis-19022.c273.us-east-1-2.ec2.redns.redis-cloud.com:19022
#Od1yizh5yilrPQCkCNcab52la5QfKQjl
hostname = 'redis-12907.c10.us-east-1-2.ec2.redns.redis-cloud.com'
portnumber = 12907
password = 'RuaoyvIP3M534KzoqzbJUWsWUaMw4UNX'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)

In [28]:
r.ping()

True

In [29]:
r.keys()

[b'academy:register']

# Load the data (numpy zip)

In [30]:
np_file =  np.load('dataframe_students_teacher.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values, columns= col_names)
df


,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ..."
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613..."
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999..."
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597..."
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732..."


In [31]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Scarlett Johansson@Student


In [32]:
records = df[['name_role', 'Facial_Features']].to_dict(orient= 'records')
len(records)

5

In [33]:
records[0]

{'name_role': 'Angelina Jolie@Student',
 'Facial_Features': array([ 4.32431698e-01,  1.07497501e+00, -1.28517359e-01, -1.83342099e+00,
        -7.27894962e-01, -1.23478580e+00, -2.34371498e-01, -3.73195887e-01,
        -6.96727574e-01, -3.52599770e-01, -1.86734408e-01, -8.56367767e-01,
        -7.25917757e-01,  1.59811819e+00,  1.89431876e-01, -8.71563181e-02,
        -8.61443207e-02,  9.60008949e-02, -4.31728780e-01,  5.76174371e-02,
         1.06118068e-01, -1.08412838e+00, -3.68282855e-01, -3.46593767e-01,
         5.47743857e-01, -3.21061790e-01,  1.65757394e+00,  3.66954863e-01,
         9.40257430e-01,  1.18212664e+00,  4.02751446e-01,  5.15111387e-01,
         2.07609081e+00, -1.63683343e+00, -1.61303684e-01, -9.74441692e-02,
        -7.65372336e-01,  7.55956531e-01, -1.70625043e+00,  2.01861054e-01,
        -1.64390409e+00,  5.47618628e-01,  2.28820726e-01, -3.25713366e-01,
         1.90994948e-01,  3.10903162e-01, -1.85834146e+00,  2.55886801e-02,
         1.24444917e-01, -4.5

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [34]:
for record in records:
    name_role = record['name_role']
    vector =  record['Facial_Features']

    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    # save data into redis cloud    
    r.hset('academy:register', key=name_role, value=vector_bytes)

### Registration Form

In [12]:
import cv2
from insightface.app import FaceAnalysis

In [13]:
#configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
#warning: dont set det_thresh<0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### step 1: Collect Person and Role

In [14]:
#python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role=input("""
    Please Choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """  
    )
    if role in ('1','2'):
        if role=='1':
            role= 'Student'
        else:
            role='Teacher'
        break
    else:
        print('Invalid entry try again')
        if i == 3:
            print('Exceeds maximum trials')

key = person_name + '@' + role
print('Your name = ', person_name)
print('Your role = ', role)
print('Key = ', key)




Enter your name:  Tri Thanh

    Please Choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
     1


Your name =  Tri Thanh
Your role =  Student
Key =  Tri Thanh@Student


In [16]:
person_name, role


('Tri Thanh', 'Student')

### step 2: Collect Samples

In [17]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break

    #get result from insightface model
    results = faceapp.get(frame, max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2,y2), (0, 255, 0), 1)

        #facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample >= 200:
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [22]:
len(face_embeddings)

200

In [23]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [24]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [25]:
# save key and facial bytes into redis database
r.hset('academy:register', key=key, value=x_mean_bytes)


0